In [1]:
from github import Github
import pandas as pd
import psycopg2
import psycopg2.extras

Get Data From GitHub

In [9]:
## Defining Search Parameters
search_parameter = ['Machine Learning location:Lagos', 'Machine Learning location:Nigeria', 'Machine Learning location:Rwanda']

In [10]:
def extract_user_info(search_parameter):

    token =  # <- Add your github token
    df_project = pd.DataFrame()

    for project in search_parameter:
        g = Github(token)
        users = g.search_users(project)
        for user in users:
            print(".")
            df_project = df_project.append({
                'name': user.name,
                'username': user.login,
                'bio': user.bio,
                'blog': user.blog,
                'company': user.company,
                'email': user.email,
                'followers': user.followers,
                'following': user.following,
                'id' : user.id,
                'url' : user.url,
                'location' : user.location
                }, ignore_index=True)
                
    #df_project.to_csv('project_dataset_2.csv', sep=',', encoding='utf-8', index=True)
    return(df_project)

In [ ]:
df = extract_user_info(search_parameter)

In [16]:
df.shape

(477, 11)

Data Ingestion - PostgresSQL

Saving PostgreSQL connection details

In [17]:
hostname = 'localhost'
database = 'data_ingest'
username = 'postgres'
pwd = 'haanuel'
port_id = 5432

Pipeline for Ingestion.
- Connect to Postgres SQL DataBase
- Create a table in the DB
- Insert values to the table

In [19]:
con = None
cur = None

try:
    with psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd,
        port = port_id ) as con:

        with con.cursor(cursor_factory= psycopg2.extras.DictCursor) as cur:
            
            cur.execute('DROP TABLE IF EXISTS github_users')
            
            create_table = ''' CREATE TABLE IF NOT EXISTS github_users (
                                name varchar(255),
                                username varchar(255),
                                bio varchar(255),
                                blog varchar(255),
                                company varchar(255),
                                email varchar(255),
                                followers int,
                                following int,
                                id varchar(255),
                                url varchar(255),
                                location varchar(255)) '''
                                
            cur.execute(create_table)
            
            insert_script = '''INSERT INTO github_users (name, username, bio, blog,
                                                        company, email, followers,
                                                        following, id, url, location) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
                                
            insert_values = list(df.to_records(index = False))
            
            for record in insert_values:
                cur.execute(insert_script, record)
                
            cur.execute('SELECT COUNT(*) FROM github_users')
            
            print('Total records loaded: ', cur.fetchall()[0])    
except Exception as error:
    print(error)
finally:
    if con is not None:
            con.close()

Total records loaded:  [477]
